In [2]:
import warnings
warnings.filterwarnings("ignore", category= UserWarning)
warnings.filterwarnings("ignore", category= FutureWarning)
warnings.filterwarnings("ignore", category= RuntimeWarning)

In [3]:
import mne
mne.set_log_level("CRITICAL")
import numpy as np
import double_dipper
from double_dipper import dataset, constants, io, ml
from double_dipper.constants import problem, strategy_prompt

In [4]:
from functools import reduce
def chain(*functions):
    def chained(args):
        for fn in functions:
            args = fn(args)
        return args
    return chained

In [5]:
def labeller(meta):
    strat = meta["strategy"]
    if strat is None: return None
    if strat.lower().startswith("fact"):        return 0
    elif strat.lower().startswith("procedure"): return 1
    else:                                       return None
divider = lambda meta: meta["epoch"]

In [6]:
def gen_dset(subjNo, split=.7):
    pairs = io.filePairs(f"cleaned/main/{subjNo}")
    dset = io.partition(divider, labeller, pairs)
    keys = sorted(dset.keys())
    X = np.concatenate([dset[k]["x"] for k in keys], axis = 0)
    Y = np.concatenate([dset[k]["y"] for k in keys], axis = 0)
    split_ind = int(len(X) * split)
    (trainX, testX) = (X[:split_ind], X[split_ind:])
    (trainY, testY) = (Y[:split_ind], Y[split_ind:])
    return (trainX, trainY, testX, testY)

In [7]:
SUBJ_NO = 10
(trX, trY, tsX, tsY) = gen_dset(SUBJ_NO, .66)

In [8]:
def problem_window(startTime, endTime):
    firstSample = problem.delay * constants.sfreq
    lastSample = strategy_prompt.delay * constants.sfreq
    return lambda X: X[...,firstSample:lastSample]

In [9]:
def bandpass_filter(l_freq, h_freq):
    return lambda X: mne.filter.filter_data(X, constants.sfreq, l_freq=l_freq, h_freq=h_freq)

In [10]:
def only_psd(fmin=0, fmax=np.inf, tmin=None, tmax=None, psd_comp=None):
    if not psd_comp:
        psd_comp = mne.time_frequency.psd_welch
    def psd_func(X):
        raw = mne.EpochsArray(X, constants.info)
        (psds, freqs) = psd_comp(raw, fmin, fmax, tmin, tmax)
        return psds
    return psd_func

In [11]:
def add_psd(*args, **kwargs):
    psd_func = only_psd(*args, **kwargs)
    return lambda X: np.concatenate([X, psd_func(X)], axis=-1)

In [40]:
def add_psd_bands(psd_comp=None):
    bands = np.array([
        [1, 3], #Delta
        [4, 7], #Theta
        [8, 11], #Alpha
        [12, 29], #Beta
        [30, 45]  #Gamma
    ])
    psd_func = only_psd(fmin=np.min(bands), fmax=np.max(bands), psd_comp=psd_comp)
    def band_func(X):
        psds = psd_func(X)
        band_powers = np.zeros([X.shape[0], bands.shape[0]])
        for (i, (l_freq, h_freq)) in enumerate(bands):
            band_powers[:, i] = np.sum(bands[:, l_freq-1:h_freq-1])
        X = flatten_end(X)
        return np.concatenate([X, band_powers], axis=-1)
    return band_func

In [13]:
def flatten_end(X):
    return X.reshape([X.shape[0], np.prod(X.shape[1:])])

In [14]:
from imblearn.over_sampling import ADASYN, SMOTE
myADASYN = lambda: ADASYN(random_state=0, n_jobs=4)
mySMOTE = lambda: SMOTE(random_state=0, n_jobs=4)

## First Pass

In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

In [16]:
feature_selectors = [
    bandpass_filter(1,32),
    chain(
        bandpass_filter(1,32),
        add_psd(1,32)
    ),
    chain(
        bandpass_filter(1,32),
        only_psd(1,32),
    ),
]
for i in range(len(feature_selectors)):
    oldFunc = feature_selectors[i]
    feature_selectors[i] = chain(problem_window(0,strategy_prompt.delay), oldFunc, flatten_end)

models = [LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis]
resamplers = [None, mySMOTE, myADASYN]

In [17]:
(inds, conf) = ml.grid_search(trX, trY, tsX, tsY, feature_selectors, resamplers, models)

feature_selector=0,resampler=0,model=0: 	precision=0.462, recall=0.429, f1=0.444
New best achieved

feature_selector=1,resampler=0,model=0: 	precision=0.333, recall=0.286, f1=0.308

feature_selector=2,resampler=0,model=0: 	precision=0.471, recall=0.571, f1=0.516
New best achieved

feature_selector=0,resampler=1,model=0: 	precision=nan, recall=0.000, f1=nan

feature_selector=1,resampler=1,model=0: 	precision=0.429, recall=0.429, f1=0.429

feature_selector=2,resampler=1,model=0: 	precision=nan, recall=0.000, f1=nan

feature_selector=0,resampler=2,model=0: 	precision=0.500, recall=0.500, f1=0.500

feature_selector=1,resampler=2,model=0: 	precision=0.360, recall=0.643, f1=0.462

feature_selector=2,resampler=2,model=0: 	precision=0.471, recall=0.571, f1=0.516

feature_selector=0,resampler=0,model=1: 	precision=nan, recall=0.000, f1=nan

feature_selector=1,resampler=0,model=1: 	precision=0.429, recall=0.429, f1=0.429

feature_selector=2,resampler=0,model=1: 	precision=nan, recall=0.000, f1=n

The PSD features seem to help, so we'll try to figure whether we need to only use them or use them with temporal features.

## Second Pass

In [18]:
feature_selectors = [
    chain(
        bandpass_filter(1,32),
        add_psd(1,32)
    ),
    chain(
        bandpass_filter(1,32),
        only_psd(1,32),
    ),
    chain(
        bandpass_filter(1,32),
        add_psd()
    ),
    chain(
        bandpass_filter(1,32),
        only_psd(),
    ),
]
for i in range(len(feature_selectors)):
    oldFunc = feature_selectors[i]
    feature_selectors[i] = chain(problem_window(0,strategy_prompt.delay), oldFunc, flatten_end)

models = [LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis]
resamplers = [None, mySMOTE, myADASYN]

In [19]:
(inds, conf) = ml.grid_search(trX, trY, tsX, tsY, feature_selectors, resamplers, models)

feature_selector=0,resampler=0,model=0: 	precision=0.500, recall=0.500, f1=0.500
New best achieved

feature_selector=1,resampler=0,model=0: 	precision=0.360, recall=0.643, f1=0.462

feature_selector=2,resampler=0,model=0: 	precision=0.471, recall=0.571, f1=0.516
New best achieved

feature_selector=3,resampler=0,model=0: 	precision=nan, recall=0.000, f1=nan

feature_selector=0,resampler=1,model=0: 	precision=0.429, recall=0.429, f1=0.429

feature_selector=1,resampler=1,model=0: 	precision=nan, recall=0.000, f1=nan

feature_selector=2,resampler=1,model=0: 	precision=0.364, recall=0.571, f1=0.444

feature_selector=3,resampler=1,model=0: 	precision=0.391, recall=0.643, f1=0.486

feature_selector=0,resampler=2,model=0: 	precision=0.333, recall=0.571, f1=0.421

feature_selector=1,resampler=2,model=0: 	precision=0.333, recall=0.071, f1=0.118

feature_selector=2,resampler=2,model=0: 	precision=0.318, recall=0.500, f1=0.389

feature_selector=3,resampler=2,model=0: 	precision=0.333, recall=0.071

Not restricting the PSD range doesn't make much of a difference, so going forward we restrict the range.

## Third Pass

In [20]:
feature_selectors = [
    chain(
        bandpass_filter(1,32),
        add_psd(1,32)
    ),
    chain(
        bandpass_filter(1,32),
        only_psd(1,32),
    ),
    chain(
        bandpass_filter(1, 40),
        add_psd(1, 32),
    ),
    chain(
        bandpass_filter(1, 40),
        only_psd(1,32)
    ),
    chain(
        bandpass_filter(1, 40),
        add_psd(1, 40),
    ),
    chain(
        bandpass_filter(1, 40),
        only_psd(1,40)
    )
]
for i in range(len(feature_selectors)):
    oldFunc = feature_selectors[i]
    feature_selectors[i] = chain(problem_window(0,strategy_prompt.delay), oldFunc, flatten_end)

models = [LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis]
resamplers = [None, mySMOTE, myADASYN]

In [21]:
(inds, conf) = ml.grid_search(trX, trY, tsX, tsY, feature_selectors, resamplers, models)

feature_selector=0,resampler=0,model=0: 	precision=0.500, recall=0.500, f1=0.500
New best achieved

feature_selector=1,resampler=0,model=0: 	precision=0.360, recall=0.643, f1=0.462

feature_selector=2,resampler=0,model=0: 	precision=0.471, recall=0.571, f1=0.516
New best achieved

feature_selector=3,resampler=0,model=0: 	precision=nan, recall=0.000, f1=nan

feature_selector=4,resampler=0,model=0: 	precision=0.429, recall=0.429, f1=0.429

feature_selector=5,resampler=0,model=0: 	precision=nan, recall=0.000, f1=nan

feature_selector=0,resampler=1,model=0: 	precision=0.364, recall=0.571, f1=0.444

feature_selector=1,resampler=1,model=0: 	precision=0.391, recall=0.643, f1=0.486

feature_selector=2,resampler=1,model=0: 	precision=0.333, recall=0.571, f1=0.421

feature_selector=3,resampler=1,model=0: 	precision=0.333, recall=0.071, f1=0.118

feature_selector=4,resampler=1,model=0: 	precision=0.318, recall=0.500, f1=0.389

feature_selector=5,resampler=1,model=0: 	precision=0.333, recall=0.071

We can probably stick to using the same PSD range as the filter range without worrying about the consequences. It also doesn't seem like expanding the filter range helped much. Now what if we shrink the filter range . . .

## Fourth Pass

In [22]:
feature_selectors = [
    chain(
        bandpass_filter(1,16),
        add_psd(1,16)
    ),
    chain(
        bandpass_filter(1,16),
        only_psd(1,16),
    ),
    chain(
        bandpass_filter(1,32),
        add_psd(1,32)
    ),
    chain(
        bandpass_filter(1,32),
        only_psd(1,32),
    ),
]
for i in range(len(feature_selectors)):
    oldFunc = feature_selectors[i]
    feature_selectors[i] = chain(problem_window(0,strategy_prompt.delay), oldFunc, flatten_end)

models = [LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis]
resamplers = [None, mySMOTE, myADASYN]

In [23]:
(inds, conf) = ml.grid_search(trX, trY, tsX, tsY, feature_selectors, resamplers, models)

feature_selector=0,resampler=0,model=0: 	precision=0.462, recall=0.429, f1=0.444
New best achieved

feature_selector=1,resampler=0,model=0: 	precision=0.438, recall=0.500, f1=0.467
New best achieved

feature_selector=2,resampler=0,model=0: 	precision=0.471, recall=0.571, f1=0.516
New best achieved

feature_selector=3,resampler=0,model=0: 	precision=nan, recall=0.000, f1=nan

feature_selector=0,resampler=1,model=0: 	precision=0.375, recall=0.429, f1=0.400

feature_selector=1,resampler=1,model=0: 	precision=nan, recall=0.000, f1=nan

feature_selector=2,resampler=1,model=0: 	precision=0.286, recall=0.429, f1=0.343

feature_selector=3,resampler=1,model=0: 	precision=0.250, recall=0.357, f1=0.294

feature_selector=0,resampler=2,model=0: 	precision=0.286, recall=0.429, f1=0.343

feature_selector=1,resampler=2,model=0: 	precision=nan, recall=0.000, f1=nan

feature_selector=2,resampler=2,model=0: 	precision=0.286, recall=0.429, f1=0.343

feature_selector=3,resampler=2,model=0: 	precision=nan, 

## Fifth Pass

In [41]:
feature_selectors = [
    chain(
        bandpass_filter(1,45),
        add_psd_bands()
    ),
]
for i in range(len(feature_selectors)):
    oldFunc = feature_selectors[i]
    feature_selectors[i] = chain(problem_window(0,strategy_prompt.delay), oldFunc, flatten_end)

models = [LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis]
resamplers = [None, mySMOTE, myADASYN]

In [42]:
(inds, conf) = ml.grid_search(trX, trY, tsX, tsY, feature_selectors, resamplers, models)

feature_selector=0,resampler=0,model=0: 	precision=0.467, recall=0.500, f1=0.483
New best achieved

feature_selector=0,resampler=1,model=0: 	precision=0.348, recall=0.571, f1=0.432

feature_selector=0,resampler=2,model=0: 	precision=0.375, recall=0.429, f1=0.400

feature_selector=0,resampler=0,model=1: 	precision=nan, recall=0.000, f1=nan

feature_selector=0,resampler=1,model=1: 	precision=0.438, recall=0.500, f1=0.467

feature_selector=0,resampler=2,model=1: 	precision=nan, recall=0.000, f1=nan

